In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.ensemble import RandomForestClassifier
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


Here we can see where we store the competition data,to load the files into my notebook.

In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


We start by reading the data from train.csv and printing the first five rows using the .head method

In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


We also read the test.csv which we are going to test it

In [4]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

.columns will print for us the columns we have in the train.csv 

In [5]:
tit_num=train_data[[ 'Age', 'SibSp','Parch', 'Fare']]
tit_cat=train_data[['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Ticket','Cabin', 'Embarked']]

I did this step to divide the columns where there are numerical and categorial.

In [6]:
from sklearn.model_selection import train_test_split

my_features= ["Pclass","Sex","Age","SibSp", "Parch","Embarked","Fare"]
train_data.drop(['PassengerId','Ticket','Cabin','Name'], axis=1,inplace=True)
X=train_data[my_features]
Y = train_data["Survived"]
test_data.drop(['Ticket','Cabin','Name'], axis=1,inplace=True)
Z=test_data[my_features]

print(train_data)

     Survived  Pclass     Sex   Age  SibSp  Parch     Fare Embarked
0           0       3    male  22.0      1      0   7.2500        S
1           1       1  female  38.0      1      0  71.2833        C
2           1       3  female  26.0      0      0   7.9250        S
3           1       1  female  35.0      1      0  53.1000        S
4           0       3    male  35.0      0      0   8.0500        S
..        ...     ...     ...   ...    ...    ...      ...      ...
886         0       2    male  27.0      0      0  13.0000        S
887         1       1  female  19.0      0      0  30.0000        S
888         0       3  female   NaN      1      2  23.4500        S
889         1       1    male  26.0      0      0  30.0000        C
890         0       3    male  32.0      0      0   7.7500        Q

[891 rows x 8 columns]


Here we are in the training phase first step to put the x and y values. my_features are the x values which are the data i want to keep and work with while the y is the output that i want to discover.
There are values that i will drop that won't help in my decision at the end or won't make a difference. At each step i will do every thing in the train_data and test_data.
Conclusion will be a table with the determined features and without the dropped ones.

In [7]:
train_data['Age'].fillna(train_data['Age'].mean(), inplace = True)
print(train_data['Age'])
print("The mean is= ",train_data['Age'].mean())
test_data['Age'].fillna(test_data['Age'].mean(), inplace = True)
print(test_data['Age'])
print("The mean is= ",test_data['Age'].mean())

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64
The mean is=  29.69911764705882
0      34.50000
1      47.00000
2      62.00000
3      27.00000
4      22.00000
         ...   
413    30.27259
414    39.00000
415    38.50000
416    30.27259
417    30.27259
Name: Age, Length: 418, dtype: float64
The mean is=  30.272590361445783


As we can see in the tables at the output of the previous cell that their are NAN values in the age column which won't be right to be left like that so i used the method fillna that fills all the NAN values by the mean value. 
Conclusion: All NAN values will be replaced with the mean value(which is calculated) and the tables will be changed in the train and test.

In [8]:
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace = True)
print(train_data['Fare'])
test_data['Fare'].fillna(test_data['Fare'].mean(), inplace = True)
print(test_data['Fare'])

0       7.2500
1      71.2833
2       7.9250
3      53.1000
4       8.0500
        ...   
886    13.0000
887    30.0000
888    23.4500
889    30.0000
890     7.7500
Name: Fare, Length: 891, dtype: float64
0        7.8292
1        7.0000
2        9.6875
3        8.6625
4       12.2875
         ...   
413      8.0500
414    108.9000
415      7.2500
416      8.0500
417     22.3583
Name: Fare, Length: 418, dtype: float64


Same thing happend here as there was a NAN value in the Fare column. so it will be changed in the train and test.

In [9]:
train_data['Embarked'].fillna("unavailable",inplace = True)
print(train_data['Embarked'])
test_data['Embarked'].fillna("unavailable",inplace = True)
print(test_data['Embarked'])

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object
0      Q
1      S
2      Q
3      S
4      S
      ..
413    S
414    C
415    S
416    S
417    C
Name: Embarked, Length: 418, dtype: object


Here there where empty cells that contained no values. Conlusion: i replaced them with unavailable using the method fillna.

In [10]:
from sklearn import preprocessing
train_data['Embarked']= preprocessing.LabelEncoder().fit_transform(train_data['Embarked'])
train_data['Sex']= preprocessing.LabelEncoder().fit_transform(train_data['Sex'])
train_data[['Embarked','Sex']].info()
print(train_data)

test_data['Embarked']= preprocessing.LabelEncoder().fit_transform(test_data['Embarked'])
test_data['Sex']= preprocessing.LabelEncoder().fit_transform(test_data['Sex'])
test_data[['Embarked','Sex']].info()
X=train_data[my_features]
Y = train_data["Survived"]
print(test_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Embarked  891 non-null    int64
 1   Sex       891 non-null    int64
dtypes: int64(2)
memory usage: 14.0 KB
     Survived  Pclass  Sex        Age  SibSp  Parch     Fare  Embarked
0           0       3    1  22.000000      1      0   7.2500         2
1           1       1    0  38.000000      1      0  71.2833         0
2           1       3    0  26.000000      0      0   7.9250         2
3           1       1    0  35.000000      1      0  53.1000         2
4           0       3    1  35.000000      0      0   8.0500         2
..        ...     ...  ...        ...    ...    ...      ...       ...
886         0       2    1  27.000000      0      0  13.0000         2
887         1       1    0  19.000000      0      0  30.0000         2
888         0       3    0  29.699118      1      2  23.4500         2

Here we used the LabelEncoder method to change the non-numerical labels to be numerical labels.Conclusion: (Embarked,Sex) which contain non-numerical data will be changed to numbers that are fixed. Example: male=1 and female=0.( Did them in both test and train).

In [11]:
train_x,valid_x,train_y,valid_y=train_test_split(X, Y,test_size=0.2, random_state=42)

The train_test_split is a function in Sklearn model selection that splits data arrays into two subsets, the first is for training data and the second for testing data. We give it parameters X and Y and the test_size and random state. It will return four things (train_x : The training part of the first sequence, valid_x : The test part of the first sequence, train_y : The training part of the second sequence,valid_y : The test part of the second sequence )

In [12]:
#training
#validation
#predection
tit_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
tit_model.fit(train_x,train_y)
titanic_score=tit_model.score(valid_x,valid_y)
print(titanic_score)
all_predictions = tit_model.predict(test_data[my_features])
mypred_output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': all_predictions})
mypred_output.to_csv('final_conclusion.csv', index=False)

0.8156424581005587


The last phase is the prediction phase, i used the random forest classifier as my model and i calculated the score which is the accuracy. And at the end using predict method, my output was the final_conclusion.csv which contained the passenger id and if he/she survived or not.